# Parte 3
Elaboración de autoencoders

Acá se importan las librerías necesarias. También se importa el dataset MNIST.

También cargamos el dataset Fashion MNIST y lo concatenamos con el dataset MNIST.
Luego de esto se mezclan los datos de entrenamiento y de prueba.
Se redimensionan los datos y se normalizan.

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

#import fashion_mnist
from tensorflow.keras.datasets import fashion_mnist, mnist
(x_train_fashion, _), (x_test_fashion, _) = fashion_mnist.load_data()
(x_train_mnist, _), (x_test_mnist, _) = mnist.load_data()

# merge datasets
x_train = np.concatenate((x_train_fashion, x_train_mnist), axis=0)
x_test = np.concatenate((x_test_fashion, x_test_mnist), axis=0)

x_train = x_train_mnist
x_test = x_test_mnist

# shuffle dataset
np.random.shuffle(x_train)
np.random.shuffle(x_test)

# reshape dataset
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

# normalize dataset
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

gpu_available = tf.config.list_physical_devices('GPU')
    
print(gpu_available)


In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(x_train[i]))
    plt.gray()
plt.show()

# Autoencoder

Para crear el autoencoder primero generaremos ruido y lo uniremos a las imágenes originales.

In [ ]:
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
plt.show()

# Creación del modelo

Se crea el modelo con una capa de entrada, una capa convolucional, una capa de pooling, una capa convolucional y una capa de pooling.

In [ ]:

class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()

# Compilación del modelo

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

# Entrenamiento del modelo

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                validation_data=(x_test_noisy, x_test))

In [ ]:
autoencoder.encoder.summary()

In [ ]:
autoencoder.decoder.summary()

Veamos cómo se ven las imágenes después de pasar por el autoencoder

In [ ]:
encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()

Observamos que las imágenes se ven bastante bien después de pasar por el autoencoder.

Ahora para realizar la última prueba, se realizarán pruebas con imagenes con ruido extremo.

In [ ]:
extreme_noise_factor = 0.75
x_train_noisy = x_train + extreme_noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + extreme_noise_factor * tf.random.normal(shape=x_test.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

Utilizaremos el autoencoder para intentar eliminar el ruido de las imágenes.

In [ ]:
encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)